In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).map(
    lambda image, label: (tf.image.resize_with_crop_or_pad(image, image_shape[0]+6, image_shape[1]+6), label)
).map(
    lambda image, label: (tf.image.random_crop(image, (image_shape[0], image_shape[1], image_shape[2])), label)
).map(
    lambda image, label: (tf.image.random_contrast(image, lower=0.2, upper=1.0), label)
).shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [9]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [10]:
layers = tf.keras.layers

In [11]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(10, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [12]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [13]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [15]:
model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

Epoch 1/200
176/176 [==============================] - 13s 76ms/step - loss: 2.3026 - accuracy: 0.1010 - val_loss: 2.3025 - val_accuracy: 0.0930
Epoch 2/200
176/176 [==============================] - 6s 33ms/step - loss: 2.3025 - accuracy: 0.1071 - val_loss: 2.3025 - val_accuracy: 0.0930
Epoch 3/200
176/176 [==============================] - 8s 48ms/step - loss: 2.3025 - accuracy: 0.1031 - val_loss: 2.3025 - val_accuracy: 0.0930
Epoch 4/200
176/176 [==============================] - 5s 31ms/step - loss: 2.3024 - accuracy: 0.1026 - val_loss: 2.3025 - val_accuracy: 0.0930
Epoch 5/200
176/176 [==============================] - 5s 31ms/step - loss: 2.3024 - accuracy: 0.1025 - val_loss: 2.3024 - val_accuracy: 0.0930
Epoch 6/200
176/176 [==============================] - 5s 31ms/step - loss: 2.3024 - accuracy: 0.1065 - val_loss: 2.3024 - val_accuracy: 0.0930
Epoch 7/200
176/176 [==============================] - 5s 31ms/step - loss: 2.3023 - accuracy: 0.1043 - val_loss: 2.3024 - val_accuracy

176/176 [==============================] - 5s 30ms/step - loss: 0.0072 - accuracy: 0.9977 - val_loss: 0.8677 - val_accuracy: 0.8740
Epoch 114/200
176/176 [==============================] - 5s 30ms/step - loss: 0.0076 - accuracy: 0.9975 - val_loss: 0.8748 - val_accuracy: 0.8746
Epoch 115/200
176/176 [==============================] - 5s 30ms/step - loss: 0.0065 - accuracy: 0.9977 - val_loss: 0.8865 - val_accuracy: 0.8758
Epoch 116/200
176/176 [==============================] - 5s 31ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.8970 - val_accuracy: 0.8770
Epoch 117/200
176/176 [==============================] - 5s 30ms/step - loss: 0.0065 - accuracy: 0.9978 - val_loss: 0.8985 - val_accuracy: 0.8756
Epoch 118/200
176/176 [==============================] - 5s 31ms/step - loss: 0.0062 - accuracy: 0.9980 - val_loss: 0.9152 - val_accuracy: 0.8736
Epoch 119/200
176/176 [==============================] - 5s 30ms/step - loss: 0.0063 - accuracy: 0.9979 - val_loss: 0.9069 - val_accuracy:

In [15]:
model.load_weights('./model_weights/vgg16_train=all_ce.h5')

In [16]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[1.0804424285888672, 0.8761000633239746]

In [34]:
# model.save('./model_weights/simple_cnn_train=all_ce.h5')

In [ ]:
# with pooling, acc can reach 72.2%
# w.o. pooling, acc is at most 67%

In [18]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[1.726710021495819, 0.78125]

In [19]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[1.3806814041454345, 0.8310547]

In [20]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========MSE============


[1.373602248262614, 0.84033203]

In [22]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[1.6405316591262817, 0.8046875]

In [ ]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

In [ ]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

In [ ]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

In [ ]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)